In [161]:
## Reading the dataset
import pandas as pd
data = pd.read_csv('comments.csv',usecols=['Text','IsToxic'],encoding='utf-8')

In [162]:
data.head()

,Text,IsToxic
0,If only people would just take a step back and...,False
1,Law enforcement is not trained to shoot to app...,True
2,\nDont you reckon them 'black lives matter' ba...,True
3,There are a very large number of people who do...,False
4,"The Arab dude is absolutely right, he should h...",False


In [163]:
data.shape

(1000, 2)

In [164]:
## Check for nulls and duplicated
print(data.isnull().sum())
print(data.duplicated().sum())

Text       0
IsToxic    0
dtype: int64
3


In [165]:
print(data[data.duplicated()])

              Text  IsToxic
657  run them over     True
677  run them over     True
699  RUN THEM OVER     True


In [166]:
## Drop duplicated
data = data.drop_duplicates()

In [167]:
## Check value counts for IsToxic
print(data['IsToxic'].value_counts())

IsToxic
False    538
True     459
Name: count, dtype: int64


In [168]:
## Dependent and Independent Features
x = data['Text']
y = data['IsToxic']

In [169]:
## Lowercasing the text
x = list(map(str.lower, x))

In [170]:
## Text Preproccessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 4881


In [171]:
x = tokenizer.texts_to_sequences(x)
max_len = max([len(sent) for sent in x])

In [172]:
max_len

815

In [173]:
## Apply padding
x = pad_sequences(x, maxlen=max_len)

In [174]:
y  = y.astype(int) # Convert to integer type

In [175]:
## Train Test Split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [192]:
## Implementing LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(32,return_sequences=False, kernel_regularizer=regularizers.l2(0.001))))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None,max_len))

In [193]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 815, 100)       │       488,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        34,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 522,213 (1.99 MB)

 Trainable params: 522,213 (1.99 MB)

 Non-trainable params: 0 (0.00 B)

In [194]:
## Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [195]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), callbacks=[early_stopping])

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 770ms/step - accuracy: 0.5319 - loss: 0.8933 - val_accuracy: 0.4700 - val_loss: 0.8512
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 643ms/step - accuracy: 0.5472 - loss: 0.8100 - val_accuracy: 0.4700 - val_loss: 0.7867
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 705ms/step - accuracy: 0.6828 - loss: 0.7079 - val_accuracy: 0.6450 - val_loss: 0.7017
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 452ms/step - accuracy: 0.8718 - loss: 0.5338 - val_accuracy: 0.6600 - val_loss: 0.6434
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 444ms/step - accuracy: 0.9297 - loss: 0.3189 - val_accuracy: 0.6950 - val_loss: 0.6140


In [203]:
sent = "This post is pathetic, just like you."
seq = tokenizer.texts_to_sequences([sent])
seq = pad_sequences(seq, maxlen=max_len)
pred = model.predict(seq)
if pred[0][0] > 0.4:
    print("Toxic Comment")
else:
    print("Non-Toxic Comment")

print(pred[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Toxic Comment
0.500062


In [211]:
## Saving the model and tokenizer
model.save('model.h5')

In [212]:
import pickle

with open("tokenizer.pickle", "wb") as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)
